# Homework 3

In [1]:
import numpy as np
from numpy.linalg import norm
from scipy.linalg import solve, eig

---

## 6.

In [2]:
data = np.array([(0, 3), (1, 5), (2, -1), (3, 2)], dtype=np.float32)

def p(degree, x):
    n = degree + 1
    xd, yd = data[:n].T
    c = yd.copy()
    for i in range(1, n + 1):
        for j in range(n - 1, i - 1, -1):
            c[j] = (c[j] - c[j-1])/(xd[j] - xd[j-i])
    r = c[0]
    for j in range(1, n):
        r += c[j]*np.prod(x - xd[:j])
    return r

In [3]:
for degree in (1, 2, 3):
    r = p(degree, 0.05)
    print('Degree %i: p(%.2f) = %.6f' % (degree, 0.05, r))

Degree 1: p(0.05) = 3.100000
Degree 2: p(0.05) = 3.290000
Degree 3: p(0.05) = 3.552438


----

## 7.

$$
\int_{0.5}^{1}x^4 dx = \frac{x^5}{5}\big|_{0.5}^1 = \frac{1 - 0.5^{5}}{5} \approx 0.19375
$$

In [4]:
def trapz(f, a, b):
    return 0.5*(f(b) + f(a))*(b - a)

In [5]:
exact = 0.19375
a = 0.5
b = 1.0
f = lambda x: x**4
r = trapz(f, a, b)
relerr = 100*abs(r - exact)/exact
print('Result: %.5f' % r)
print('Relative Error: %.5f%%' % relerr)

Result: 0.26562
Relative Error: 37.09677%


## 8.

$$
\int_1^2 x \ln x ~dx \approx 0.636294
$$

In [6]:
exact = 0.636294
n = 3
a = 1.0
b = 2.0
h = (b - a)/n
points = [i*h + a for i in range(n + 1)]
points

[1.0, 1.3333333333333333, 1.6666666666666665, 2.0]

In [7]:
f = lambda x: np.log(x)*x
r = 0
for i in range(n):
    r += trapz(f, points[i], points[i + 1])

In [8]:
relerr = 100*abs(r - exact)/exact
print('Result: %.5f' % r)
print('Relative Error: %.5f%%' % relerr)

Result: 0.64270
Relative Error: 1.00673%


## 9.

In [9]:
A = np.array([
    [-4, 14, 0],
    [-5, 13, 0],
    [-1, 0, 0],
], dtype=np.float32)
x0 = np.ones(3, dtype=np.float32)
tol = 1e-16

In [10]:
eigvals, eigvecs = eig(A)
s = np.argsort(eigvals)
eigvals, eigvecs = eigvals[s], eigvecs[:, s]
print('Eigenvalue : Eigenvector pairs')
print('\n'.join(
    ['%s : %s' % (eigvals[i], eigvecs[:, i])
     for i in range(len(eigvals))]
))

Eigenvalue : Eigenvector pairs
0j : [ 0.  0.  1.]
(3+0j) : [ 0.85714287  0.42857143 -0.28571424]
(6+0j) : [ 0.80635154  0.57596534 -0.13439195]


In [11]:
def pwrmethod(x):
    y = A.dot(x)
    return y/norm(y)

def rayleigh(x):
    xs = np.conj(x)
    return (xs.dot(A).dot(x))/(xs.dot(x))

In [12]:
def q9(x0):
    x = x0
    lam = np.nan
    for i in range(100):
        xn = pwrmethod(x)
        lamn = rayleigh(xn)
        if abs(lamn - lam) < tol:
            break
        x, lam = xn, lamn
    else:
        print('Maximum number of iterations reached.')
    return x, lamn, i

In [13]:
x, lam, iters = q9(x0)
print('x = %s' % x)
print('lambda = %.16f' % lamn)
print('number of iterations : %i' % iters)

x = [ 0.80635148  0.57596534 -0.1343919 ]


NameError: name 'lamn' is not defined

## 10.

In [14]:
def q10(x0):
    x = x0
    lam = np.nan
    for i in range(100):
        x1 = pwrmethod(x)
        x2 = pwrmethod(x1)
        # Aitken's method
        delta = x1 - x
        delta2 = x - 2*x1 + x2 + tol  # Don't want to divide by zero.
        aitken = x - delta**2/delta2
        lamn = rayleigh(x)
        if abs(lamn - lam) < tol:
            break
        x, lam = aitken, lamn
    else:
        print('Maximum number of iterations reached.')
    return x, lamn, i

In [15]:
x, lam, iters = q10(x0)
print('x = %s' % x)
print('lambda = %.16f' % lam)
print('number of iterations : %i' % iters)

x = [ 0.80635148  0.57596534 -0.13439192]
lambda = 6.0000000000000000
number of iterations : 6


In [16]:
t9 = %timeit -o q9(x0)

1000 loops, best of 3: 355 µs per loop


In [17]:
t10 = %timeit -o q10(x0)

1000 loops, best of 3: 237 µs per loop


In [18]:
ratio = 100*(t10.best/t9.best)
print("Aitken's delta squared method makes the computation take\n"
      "%.1f%% of the time." % ratio)

Aitken's delta squared method makes the computation take
66.8% of the time.


## 11.

In [19]:
I = np.eye(len(A))
def invpwrmethod(x, mu):
    y = solve(A - mu*I, x)
    return y/norm(y)

In [20]:
def q11(x0, mu):
    x = x0
    lam = np.nan
    for i in range(100):
        xn = invpwrmethod(x, mu)
        lamn = rayleigh(xn)
        if abs(lamn - lam) < tol:
            break
        x, lam = xn, lamn
    else:
        print('Maximum number of iterations reached.')
    return x, lamn, i

In [21]:
x, lam, iters = q11(x0, 3.5)
print('x = %s' % x)
print('lambda = %.16f' % lam)
print('number of iterations : %i' % iters)

x = [-0.85714286 -0.42857143  0.28571429]
lambda = 2.9999999999999960
number of iterations : 26
